In [20]:
import tkinter as tk
from tkinter import ttk
from tkinter import filedialog
from tkinter import messagebox
import tkinter.font as tkf
import sqlite3
import datetime

from libvin.decoding import Vin

# DB Connection

In [21]:
DB_file = 'BraeswoodDB.sqlite'
conn = sqlite3.connect(DB_file)
cur = conn.cursor()

In [22]:
sqlstr = 'SELECT * FROM congenital'

spreadsheet = cur.execute(sqlstr)

headers_db = []

for unit in spreadsheet.description:
    headers_db.append(unit[0])

combination = []

for row_tuple in spreadsheet:
    combination.append(row_tuple)

## Tabulate Functions

In [23]:
def ExtractID(iden): 
    sqlstr = 'SELECT * FROM VIN WHERE congenital.id = ?'
    cur.execute(sqlstr, (iden,))
    rowSelected = cur.fetchone()
   
    item = {}    
    for i in range(len(rowSelected)):
        item[headers_db[i]] = rowSelected[i]
   
    display_in_text(item)

In [24]:
def OnDoubleClick(event):
    global idglb
    try:
        item = table.selection()[0]
        value = table.item(item, 'values')    
        iden = value[0]
        ExtractID(iden)     
        idglb = iden
        
    except:
        pass

In [25]:
def sortby(tree, col, descending):
    """sort tree contents when a column header is clicked on"""
    # grab values to sort
    data = [(tree.set(child, col), child) for child in tree.get_children('')]
    # if the data to be sorted is numeric change to float
    #data =  change_numeric(data)
    # now sort the data in place
    data.sort(reverse=descending)
    for ix, item in enumerate(data):
        tree.move(item[1], '', ix)
    # switch the heading so it will sort in the opposite direction
    tree.heading(col, command=lambda col=col: sortby(tree, col, int(not descending)))

In [26]:
def refreshDB():
    global conn, cur, desc, headers, combination
    conn.close()
    conn = sqlite3.connect(DB_file)
    cur = conn.cursor()

    cur.execute('SELECT * FROM congenital')
    
    sqlstr = 'SELECT * FROM congenital ORDER BY congenital.id'
    
    spreadsheet = cur.execute(sqlstr)

    combination = []
    
    for row_tuple in spreadsheet:
        combination.append(list(row_tuple[:6]) + list(row_tuple[6]) + list(row_tuple[7:]))

In [27]:
def display_in_text():
    pass

In [28]:
def display_in_table(combination):
    for row in combination:
        table.insert("", "end", "", values=row)
    num = str(len(combination))
    text_num.delete('1.0', tk.END)
    text_num.insert('1.0', num)

# Main Sream

In [29]:
root = tk.Tk()
root.geometry("{0}x{1}+0+0".format(root.winfo_screenwidth(), root.winfo_screenheight()))
#root.attributes('-fullscreen', True)
root.title('Braeswood')
#root.iconbitmap('GessnerIcon.ico')

''

## Table Control

In [30]:
headers = ['id', 'PatientID', 'Name', 'NameCN', 'Initial', 'Gender',
           'CitizenID', 'Race', 'OperationDate', 'Operator', 'Assistant', 
           'DiagnosisPre', 'DiagnosisPost', 'Murmur', 'ECG', 'X-Ray', 
           'Echo', 'Occluder', 'OccluderBrand', 'OccluderSize', 'Instruments', 
           'ComplicationPost', 'Recorder', 'Telephone', 'Comments']
header_widths = [10, 80, 80, 50, 30, 100,
                 20, 20, 50, 20, 20, 
                 20, 20, 20, 50, 20, 
                 20, 20, 20, 20, 20, 
                 20, 20, 40, 40]

In [31]:
# /////////////////Multicolumn Listbox//////////////////////////////////////////////
# ipadx and ipady are diameters.

table = ttk.Treeview(root, height="20", columns=headers, selectmode="extended")
table.pack(padx=10, pady=20, ipadx=1200, ipady=185)

i = 1
for header in headers:
    table.heading('#'+str(i), text=header.title(), anchor=tk.W, command=lambda c=header: sortby(table, c, 0))
    table.column('#'+str(i), stretch=tk.NO, minwidth=0, width=tkf.Font().measure(header.title())+header_widths[i-1])
    i+=1    
table.column('#0', stretch=tk.NO, minwidth=0, width=0)

table.bind("<Double-1>", OnDoubleClick)
#///////////////////////////////////////////////////////////////////////////////////////////

# Scrollbar////////////////////////////////////////////////////////////////////////////////////////
vsb = ttk.Scrollbar(table, orient = "vertical",  command = table.yview)
hsb = ttk.Scrollbar(table, orient = "horizontal", command = table.xview)
## Link scrollbars activation to top-level object
table.configure(yscrollcommand = vsb.set, xscrollcommand = hsb.set)
## Link scrollbar also to every columns
map(lambda col: col.configure(yscrollcommand = vsb.set, xscrollcommand = hsb.set), table)
vsb.pack(side = tk.RIGHT, fill = tk.Y)
hsb.pack(side = tk.BOTTOM, fill = tk.X) 

## Other Controls

In [32]:
root.mainloop()

In [33]:
conn.close()